# QUESTION 1

In [1]:
import pandas as pd

# Given Data
fixed_cost_in_house = 50000  # Fixed cost for in-house manufacturing
unit_variable_cost_in_house = 130  # Unit variable cost for in-house manufacturing
unit_cost_outsource = 180  # Unit cost for outsourcing

# Levels of demand
demand_levels = [800, 1000, 1200, 1500]

# Calculate the total cost for in-house manufacturing and outsourcing
in_house_costs = []
outsource_costs = []

for demand in demand_levels:
    in_house_total_cost = fixed_cost_in_house + (unit_variable_cost_in_house * demand)
    outsource_total_cost = unit_cost_outsource * demand
    
    in_house_costs.append(in_house_total_cost)
    outsource_costs.append(outsource_total_cost)

# Create the payoff table as a DataFrame
payoff_data = {
    "Demand": demand_levels,
    "Manufacture": in_house_costs,
    "Outsource": outsource_costs
}

payoff_table = pd.DataFrame(payoff_data)

# Print the payoff table
print(payoff_table)


   Demand  Manufacture  Outsource
0     800       154000     144000
1    1000       180000     180000
2    1200       206000     216000
3    1500       245000     270000


Question 6

In [2]:
# Given values
insurance_cost = 300
cost_fender_bender = 3500
cost_major_accident = 17500
prob_major_accident = 0.0006
prob_fender_bender = 0.0016
prob_no_accident = 1 - (prob_major_accident + prob_fender_bender)

# Expected Value without Insurance
EV_no_insurance = (0 * prob_no_accident) + (cost_fender_bender * prob_fender_bender) + (cost_major_accident * prob_major_accident)

# Rollback value for taking insurance is just the cost of the insurance
EV_take_insurance = insurance_cost

print(f"Rollback Value for 'Take Insurance': ${EV_take_insurance:.2f}")

print(f"Expected Value for 'Do Not Take Insurance': ${EV_no_insurance:.2f}")



Rollback Value for 'Take Insurance': $300.00
Expected Value for 'Do Not Take Insurance': $16.10


# QUESTION 8

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def get_input_data():
    """
    Function to get input data from the user.
    Returns a dictionary containing all necessary data for the analysis.
    """
    print("=== Snow Shovel Ordering Sensitivity Analysis ===\n")
    
    # Order quantities (must be multiples of 200 up to 3000)
    default_order_quantities = [200, 400, 1400, 1600, 2400, 2600, 3000]
    print("Default Order Quantities (units):", default_order_quantities)
    order_quantities_input = input("Enter order quantities separated by commas (Press Enter to use default): ")
    if order_quantities_input.strip() == "":
        order_quantities = default_order_quantities
    else:
        order_quantities = [int(q.strip()) for q in order_quantities_input.split(",")]
    
    # Costs and Prices
    try:
        cost_per_shovel = float(input("\nEnter the cost per shovel (default $16.00): ") or 16.00)
        selling_price = float(input("Enter the selling price per shovel (default $31.95): ") or 31.95)
        discount_price = float(input("Enter the discount price per shovel (default $10.00): ") or 10.00)
    except ValueError:
        print("Invalid input. Using default values.")
        cost_per_shovel = 16.00
        selling_price = 31.95
        discount_price = 10.00
    
    # Demand Distributions
    print("\n--- Demand Distribution for Mild Winter ---")
    mild_demands = []
    mild_probabilities = []
    num_mild = int(input("Enter the number of demand scenarios for Mild Winter (default 3): ") or 3)
    for i in range(num_mild):
        demand = int(input(f"  Enter demand quantity #{i+1} for Mild Winter: "))
        prob = float(input(f"  Enter probability for demand {demand} (as a decimal, e.g., 0.5 for 50%): "))
        mild_demands.append(demand)
        mild_probabilities.append(prob)
    
    print("\n--- Demand Distribution for Harsh Winter ---")
    harsh_demands = []
    harsh_probabilities = []
    num_harsh = int(input("Enter the number of demand scenarios for Harsh Winter (default 3): ") or 3)
    for i in range(num_harsh):
        demand = int(input(f"  Enter demand quantity #{i+1} for Harsh Winter: "))
        prob = float(input(f"  Enter probability for demand {demand} (as a decimal, e.g., 0.1 for 10%): "))
        harsh_demands.append(demand)
        harsh_probabilities.append(prob)
    
    # Validate probabilities
    if not np.isclose(sum(mild_probabilities), 1.0):
        print("\nWarning: Sum of Mild Winter probabilities does not equal 1. Normalizing...")
        total = sum(mild_probabilities)
        mild_probabilities = [p / total for p in mild_probabilities]
    
    if not np.isclose(sum(harsh_probabilities), 1.0):
        print("Warning: Sum of Harsh Winter probabilities does not equal 1. Normalizing...")
        total = sum(harsh_probabilities)
        harsh_probabilities = [p / total for p in harsh_probabilities]
    
    # Probabilities of harsh winter for sensitivity analysis
    default_p_harsh = [0.2, 0.4, 0.6, 0.8]
    p_harsh_input = input("\nEnter probabilities of harsh winter separated by commas (default 0.2,0.4,0.6,0.8): ")
    if p_harsh_input.strip() == "":
        p_harsh = default_p_harsh
    else:
        p_harsh = [float(p.strip()) for p in p_harsh_input.split(",")]
    
    data = {
        'order_quantities': order_quantities,
        'cost_per_shovel': cost_per_shovel,
        'selling_price': selling_price,
        'discount_price': discount_price,
        'mild_demands': mild_demands,
        'mild_probabilities': mild_probabilities,
        'harsh_demands': harsh_demands,
        'harsh_probabilities': harsh_probabilities,
        'p_harsh_list': p_harsh
    }
    
    return data

def calculate_expected_profit(order_qty, demands, probabilities, selling_price, discount_price, cost_per_shovel):
    """
    Calculate the expected profit for a given order quantity and demand distribution.
    
    Parameters:
    - order_qty: Ordered quantity
    - demands: List of possible demand quantities
    - probabilities: Corresponding probabilities for each demand
    - selling_price: Selling price per unit
    - discount_price: Discount price per unit for unsold shovels
    - cost_per_shovel: Cost per unit
    
    Returns:
    - Expected profit for the given order quantity
    """
    expected_profit = 0.0
    for demand, prob in zip(demands, probabilities):
        sold_units = min(order_qty, demand)
        unsold_units = max(order_qty - demand, 0)
        revenue = sold_units * selling_price + unsold_units * discount_price
        profit = revenue - (order_qty * cost_per_shovel)
        expected_profit += prob * profit
    return expected_profit

def perform_sensitivity_analysis(data):
    """
    Perform sensitivity analysis over varying probabilities of a harsh winter.
    
    Parameters:
    - data: Dictionary containing all input data
    
    Returns:
    - DataFrame containing optimal order quantities and expected profits for each p_harsh
    """
    results = []
    for p_harsh in data['p_harsh_list']:
        p_mild = 1 - p_harsh
        # Calculate expected profit for mild and harsh winters
        e_profit_mild = calculate_expected_profit(
            order_qty=0,  # Placeholder, will be computed for each order quantity
            demands=data['mild_demands'],
            probabilities=data['mild_probabilities'],
            selling_price=data['selling_price'],
            discount_price=data['discount_price'],
            cost_per_shovel=data['cost_per_shovel']
        )
        e_profit_harsh = calculate_expected_profit(
            order_qty=0,  # Placeholder, will be computed for each order quantity
            demands=data['harsh_demands'],
            probabilities=data['harsh_probabilities'],
            selling_price=data['selling_price'],
            discount_price=data['discount_price'],
            cost_per_shovel=data['cost_per_shovel']
        )
        
        # For each order quantity, compute expected profit
        for Q in data['order_quantities']:
            # Expected profit under mild winter
            e_profit_mild_Q = calculate_expected_profit(
                order_qty=Q,
                demands=data['mild_demands'],
                probabilities=data['mild_probabilities'],
                selling_price=data['selling_price'],
                discount_price=data['discount_price'],
                cost_per_shovel=data['cost_per_shovel']
            )
            # Expected profit under harsh winter
            e_profit_harsh_Q = calculate_expected_profit(
                order_qty=Q,
                demands=data['harsh_demands'],
                probabilities=data['harsh_probabilities'],
                selling_price=data['selling_price'],
                discount_price=data['discount_price'],
                cost_per_shovel=data['cost_per_shovel']
            )
            # Total expected profit
            total_e_profit = p_harsh * e_profit_harsh_Q + p_mild * e_profit_mild_Q
            results.append({
                'p_harsh': p_harsh,
                'order_qty': Q,
                'expected_profit': total_e_profit
            })
    
    # Convert results to DataFrame
    df_results = pd.DataFrame(results)
    
    # Determine optimal order quantity for each p_harsh
    optimal_results = df_results.loc[df_results.groupby('p_harsh')['expected_profit'].idxmax()].reset_index(drop=True)
    
    return optimal_results

def plot_expected_profit(optimal_results):
    """
    Plot the optimal expected profit as a function of the probability of a harsh winter.
    
    Parameters:
    - optimal_results: DataFrame containing optimal order quantities and expected profits
    """
    plt.figure(figsize=(10, 6))
    plt.plot(optimal_results['p_harsh'], optimal_results['expected_profit'], marker='o', linestyle='-', color='b')
    
    for idx, row in optimal_results.iterrows():
        plt.text(row['p_harsh'], row['expected_profit'], f"Q={row['order_qty']}", fontsize=9, ha='right')
    
    plt.title('Optimal Expected Profit vs. Probability of Harsh Winter')
    plt.xlabel('Probability of Harsh Winter')
    plt.ylabel('Optimal Expected Profit ($)')
    plt.xticks(optimal_results['p_harsh'])
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def main():
    # Step 1: Get input data
    data = get_input_data()
    
    # Step 2: Perform sensitivity analysis
    optimal_results = perform_sensitivity_analysis(data)
    
    # Step 3: Display results
    print("\n=== Sensitivity Analysis Results ===")
    print(optimal_results.to_string(index=False, 
                                    formatters={'p_harsh': '{:.2%}'.format, 
                                                'expected_profit': '${:,.2f}'.format}))
    
    # Step 4: Plot the results
    plot_expected_profit(optimal_results)
    
    # Optional: Save results to CSV
    save_option = input("\nWould you like to save the results to a CSV file? (y/n): ").strip().lower()
    if save_option == 'y':
        filename = input("Enter the filename (e.g., results.csv): ").strip()
        optimal_results.to_csv(filename, index=False)
        print(f"Results saved to {filename}")

if __name__ == "__main__":
    main()


=== Snow Shovel Ordering Sensitivity Analysis ===

Default Order Quantities (units): [200, 400, 1400, 1600, 2400, 2600, 3000]

--- Demand Distribution for Mild Winter ---


ValueError: invalid literal for int() with base 10: ''